In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, ctx
import plotly.express as px
# Example Gantt data
df_gantt = pd.DataFrame({
    "bus_id": ["Bus1", "Bus1", "Bus1", "Bus1", "Bus1", "Bus2", "Bus2", "Bus2", "Bus2", "Bus2", "Bus3", "Bus3"],
    "trip_id": ["AC1", "CA2", "CS1", "CA4", "AC4", "AC2", "CA3", "CS1", "CA5", "AC5", "CA1", "AC3"],
    "dep_time_min": [555, 740, 880, 920, 1155, 675, 860, 1000, 1040, 1215, 570, 795],
    "arr_time_min": [695, 880, 930, 1060, 1295, 815, 1000, 1050, 1180, 1355, 710, 935]
})
base_date = pd.Timestamp("2025-01-01")
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
time_range = pd.date_range(base_date, base_date + pd.Timedelta("24h"), freq="10min")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
D_MAX = 350

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash import Dash, dcc, html, Input, Output, ctx
import plotly.express as px
# Example Gantt data
df_gantt = pd.DataFrame({
    "bus_id": ["Bus1", "Bus1", "Bus1", "Bus1", "Bus1", "Bus2", "Bus2", "Bus2", "Bus2", "Bus2", "Bus3", "Bus3"],
    "trip_id": ["AC1", "CA2", "CS1", "CA4", "AC4", "AC2", "CA3", "CS1", "CA5", "AC5", "CA1", "AC3"],
    "dep_time_min": [555, 740, 880, 920, 1155, 675, 860, 1000, 1040, 1215, 570, 795],
    "arr_time_min": [695, 880, 930, 1060, 1295, 815, 1000, 1050, 1180, 1355, 710, 935]
})
base_date = pd.Timestamp("2025-01-01")
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
time_range = pd.date_range(base_date, base_date + pd.Timedelta("24h"), freq="10min")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
D_MAX = 350

# -----------------------------------------------------------
# Initialize Dash App
# -----------------------------------------------------------
app = Dash(__name__)
app.title = "Electric Bus Rostering Dashboard"

instances = {"10T": "10Trips", "20T": "20Trips", "30T": "30Trips", "40T": "40Trips", "54T": "54Trips", "100T": "100Trips"}

# -----------------------------------------------------------
# Layout
# -----------------------------------------------------------
app.layout = html.Div([
    html.H2("Electric Bus Rostering Dashboard", style={"textAlign": "center"}),

    html.Div([
        # LEFT PANEL – Control Panel
        html.Div([
            html.H4("Select Instance"),
            dcc.Dropdown(
                id="instance_selector",
                options=[{"label": k, "value": k} for k in instances.keys()],
                value="10T",
                clearable=False
            ),
            html.Div(id="instance_desc", style={"marginTop": "20px", "fontStyle": "italic"})
        ],
        style={
            "width": "20%",
            "padding": "15px",
            "backgroundColor": "#f5f5f5",
            "borderRadius": "10px"
        }),

        # RIGHT PANEL – Figures
        html.Div([
            html.Div([
                html.Div([
                    dcc.Graph(id="gantt", style={"width": "48%", "display": "inline-block"}),
                    dcc.Graph(id="gantt_3cs", style={"width": "48%", "display": "inline-block"})
                ]),
                html.Hr(),
                html.H4(f"1CS Battery State of Charge (SoC)"),
                dcc.Graph(id="soc_line"),
                html.H4(f"3CS Battery State of Charge (SoC)"),
                dcc.Graph(id="soc_line_3")
            ])
        ],
        style={"width": "75%", "padding": "15px"})
    ],
    style={"display": "flex", "gap": "20px"})
])


# -----------------------------------------------------------
# CALLBACKS
# -----------------------------------------------------------

# 1️⃣ Update Gantt Charts when instance changes
@app.callback(
    Output("gantt", "figure"),
    Output("gantt_3cs", "figure"),    
    Output("soc_line", "figure", allow_duplicate=True),
    Output("soc_line_3", "figure", allow_duplicate=True),
    Input("instance_selector", "value"),
    prevent_initial_call="initial_duplicate"
)
def test_instances(case):
    print(f"CASE!!! = {case}")
    work_dir = f"./FINAL_TimeLimit/{case}rips/"
    single = pd.read_csv(f"{work_dir}CHSSA_{case}1CS_df_BEFORE.csv")
    df_gantt = single.drop(columns="Unnamed: 0")
    df_gantt['dep_time_min'] = df_gantt['dep_time']
    df_gantt['arr_time_min'] = df_gantt['arr_time']
    df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
    df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
    df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
    df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

    many = pd.read_csv(f"{work_dir}CHSSA_{case}3CS_df.csv")
    df_gantt_3 = many.drop(columns="Unnamed: 0")
    df_gantt_3.head()
    df_gantt_3['dep_time_min'] = df_gantt_3['dep_time']
    df_gantt_3['arr_time_min'] = df_gantt_3['arr_time']
    df_gantt_3["dep_time"] = base_date + pd.to_timedelta(df_gantt_3["dep_time_min"], unit="m")
    df_gantt_3["arr_time"] = base_date + pd.to_timedelta(df_gantt_3["arr_time_min"], unit="m")
    df_gantt_3['energy_required'] = df_gantt_3['arr_time_min'] - df_gantt_3['dep_time_min']
    df_gantt_3['id'] = df_gantt_3['bus_id'].apply(lambda x: f"Bus{x}")
   
    soc_data = []
    for bus_id, group in df_gantt.groupby("bus_id"):
        soc = 100
        for id, trip in group.iterrows():
            trip_id = trip['trip_id']
            if trip_id.startswith("CS"):
                soc = 0
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
                soc_data.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, 100, len(time_range))
                }))
                soc = 100
            else:
                prev_soc = soc
                soc = soc - (trip['energy_required']/D_MAX * 100)
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
                soc_data.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
                }))
    df_soc = pd.concat(soc_data)


    soc_data_3 = []
    for bus_id, group in df_gantt_3.groupby("bus_id"):
        soc = 100
        for id, trip in group.iterrows():
            trip_id = trip['trip_id']
            if trip_id.startswith("CS"):
                soc = 0
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
                soc_data_3.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, 100, len(time_range))
                }))
                soc = 100
            else:
                prev_soc = soc
                soc = soc - (trip['energy_required']/D_MAX * 100)
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
                soc_data_3.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
                }))

    df_soc_3 = pd.concat(soc_data_3)  
    ########################################## 1 CS ##############################################
    # Create base color map dynamically
    trip_ids = df_gantt["trip_id"].unique()
    color_map = {t: "#1f77b4" for t in trip_ids}  # default all blue

    # Override "CS*" trip colors
    for t in trip_ids:
        if str(t).startswith("CS"):
            color_map[t] = "#ff7f0e"

    fig_gantt = px.timeline(
        df_gantt, 
        x_start="dep_time", x_end="arr_time", 
        y="id", color="trip_id",
        title=f"Electric Bus Timetable {case}1CS",
        color_discrete_map=color_map
    )

    fig_gantt.update_layout(
        yaxis=dict(
            tickmode="array",
            tickvals=df_gantt.id.unique(),
            ticktext=[f"{i}" for i in df_gantt.id.unique()]
        )
    )

    fig_gantt.update_yaxes(autorange="reversed")

    ########################################## 3 CS ##############################################
    trip_ids = df_gantt_3["trip_id"].unique()
    color_map = {t: "#1f77b4" for t in trip_ids}  # default all blue

    # Override "CS*" trip colors
    for t in trip_ids:
        if str(t).startswith("CS"):
            color_map[t] = "#ff7f0e"

    fig_gantt_3 = px.timeline(
        df_gantt_3, 
        x_start="dep_time", x_end="arr_time", 
        y="id", color="trip_id",
        title=f"Electric Bus Timetable {case}3CS",
        color_discrete_map=color_map
    )

    fig_gantt_3.update_layout(
        yaxis=dict(
            tickmode="array",
            tickvals=df_gantt_3.id.unique(),
            ticktext=[f"{i}" for i in df_gantt_3.id.unique()]
        )
    )
    
    fig = px.line(df_soc, x="time", y="soc", color="bus_id", title=f"{case}1CS State of Charge (All Buses)")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
    fig2 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title=f"{case}3CS State of Charge (All Buses)")
    fig2.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig2.update_xaxes(title="Time of Day", tickformat="%H:%M")

    fig_gantt_3.update_yaxes(autorange="reversed")
    return fig_gantt, fig_gantt_3, fig, fig2

# 2️⃣ Update SoC Line Chart based on clicks from either Gantt
@app.callback(
    Output("soc_line", "figure"),
    Output("soc_line_3", "figure"),
    [Input("gantt", "clickData"),
    Input("gantt_3cs", "clickData"),
    Input("instance_selector", "value")]
)
def show_soc_line(clickData, clickData3cs, case):
    triggered = ctx.triggered_id
    work_dir = f"./FINAL_TimeLimit/{case}rips/"
    single = pd.read_csv(f"{work_dir}CHSSA_{case}1CS_df.csv")
    df_gantt = single.drop(columns="Unnamed: 0")
    df_gantt['dep_time_min'] = df_gantt['dep_time']
    df_gantt['arr_time_min'] = df_gantt['arr_time']
    df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
    df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
    df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
    df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

    many = pd.read_csv(f"{work_dir}CHSSA_{case}3CS_df.csv")
    df_gantt_3 = many.drop(columns="Unnamed: 0")
    df_gantt_3.head()
    df_gantt_3['dep_time_min'] = df_gantt_3['dep_time']
    df_gantt_3['arr_time_min'] = df_gantt_3['arr_time']
    df_gantt_3["dep_time"] = base_date + pd.to_timedelta(df_gantt_3["dep_time_min"], unit="m")
    df_gantt_3["arr_time"] = base_date + pd.to_timedelta(df_gantt_3["arr_time_min"], unit="m")
    df_gantt_3['energy_required'] = df_gantt_3['arr_time_min'] - df_gantt_3['dep_time_min']
    df_gantt_3['id'] = df_gantt_3['bus_id'].apply(lambda x: f"Bus{x}")
    
    soc_data = []
    for bus_id, group in df_gantt.groupby("bus_id"):
        soc = 100
        for id, trip in group.iterrows():
            trip_id = trip['trip_id']
            if trip_id.startswith("CS"):
                soc = 0
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
                soc_data.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, 100, len(time_range))
                }))
                soc = 100
            else:
                prev_soc = soc
                soc = soc - (trip['energy_required']/D_MAX * 100)
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
                soc_data.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
                }))
    df_soc = pd.concat(soc_data)

    soc_data_3 = []
    for bus_id, group in df_gantt_3.groupby("bus_id"):
        soc = 100
        for id, trip in group.iterrows():
            trip_id = trip['trip_id']
            if trip_id.startswith("CS"):
                soc = 0
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
                soc_data_3.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, 100, len(time_range))
                }))
                soc = 100
            else:
                prev_soc = soc
                soc = soc - (trip['energy_required']/D_MAX * 100)
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
                soc_data_3.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
                }))

    df_soc_3 = pd.concat(soc_data_3)  
    bus_clicked = bus3_clicked = None
    if triggered == "gantt" and clickData:
        bus_clicked = clickData["points"][0]["y"]
    elif triggered == "gantt_3cs" and clickData3cs:
        bus3_clicked = clickData3cs["points"][0]["y"]
    else:
        print(f"CLIKED DATA = {clickData}")
        fig = px.line(df_soc, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
        fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
        fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
        
        fig_3 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
        fig_3.update_yaxes(title="State of Charge (%)", range=[0, 100])
        fig_3.update_xaxes(title="Time of Day", tickformat="%H:%M") 
        return fig, fig_3
    
    if bus_clicked:
        df_selected = df_soc[df_soc["bus_id"] == bus_clicked]
        df_selected_3 = df_soc_3[df_soc_3["bus_id"] == bus_clicked]
    elif bus3_clicked:
        df_selected = df_soc[df_soc["bus_id"] == bus3_clicked]
        df_selected_3 = df_soc_3[df_soc_3["bus_id"] == bus3_clicked]
        bus_clicked = bus3_clicked
     
    
    fig = px.line(df_soc, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
    fig2 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
    fig2.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig2.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
    clickData = clickData3cs = None
    return fig, fig2

app.run(debug=False, host='0.0.0.0', port=8501)

CASE!!! = 10T
CLIKED DATA = None


In [76]:
# -----------------------------------------------------------
# Initialize Dash App
# -----------------------------------------------------------
app = Dash(__name__)
app.title = "Electric Bus Rostering Dashboard"

instances = {"10T": "10Trips", "20T": "20Trips", "30T": "30Trips", "40T": "40Trips", "54T": "54Trips", "100T": "100Trips"}

# -----------------------------------------------------------
# Layout
# -----------------------------------------------------------
app.layout = html.Div([
    html.H2("Electric Bus Rostering Dashboard", style={"textAlign": "center"}),

    html.Div([
        # LEFT PANEL – Control Panel
        html.Div([
            html.H4("Select Instance"),
            dcc.Dropdown(
                id="instance_selector",
                options=[{"label": k, "value": k} for k in instances.keys()],
                value="10T",
                clearable=False
            ),
            html.Div(id="instance_desc", style={"marginTop": "20px", "fontStyle": "italic"})
        ],
        style={
            "width": "20%",
            "padding": "15px",
            "backgroundColor": "#f5f5f5",
            "borderRadius": "10px"
        }),

        # RIGHT PANEL – Figures
        html.Div([
            html.Div([
                html.Div([
                    dcc.Graph(id="gantt", style={"width": "48%", "display": "inline-block"}),
                    dcc.Graph(id="gantt_3cs", style={"width": "48%", "display": "inline-block"})
                ]),
                html.Hr(),
                html.H4("1CS Battery State of Charge (SoC)"),
                dcc.Graph(id="soc_line"),
                html.H4("3CS Battery State of Charge (SoC)"),
                dcc.Graph(id="soc_line_3")
            ])
        ],
        style={"width": "75%", "padding": "15px"})
    ],
    style={"display": "flex", "gap": "20px"})
])


# -----------------------------------------------------------
# CALLBACKS
# -----------------------------------------------------------

# 1️⃣ Update Gantt Charts when instance changes
@app.callback(
    Output("gantt", "figure"),
    Output("gantt_3cs", "figure"),
    Input("instance_selector", "value")
)
def test_instances(case):
    print(f"CASE!!! = {case}")
    work_dir = f"./FINAL_TimeLimit/{case}rips/"
    single = pd.read_csv(f"{work_dir}CHSSA_{case}1CS_df.csv")
    df_gantt = single.drop(columns="Unnamed: 0")
    df_gantt['dep_time_min'] = df_gantt['dep_time']
    df_gantt['arr_time_min'] = df_gantt['arr_time']
    df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
    df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
    df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
    df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

    many = pd.read_csv(f"{work_dir}CHSSA_{case}3CS_df.csv")
    df_gantt_3 = many.drop(columns="Unnamed: 0")
    df_gantt_3.head()
    df_gantt_3['dep_time_min'] = df_gantt_3['dep_time']
    df_gantt_3['arr_time_min'] = df_gantt_3['arr_time']
    df_gantt_3["dep_time"] = base_date + pd.to_timedelta(df_gantt_3["dep_time_min"], unit="m")
    df_gantt_3["arr_time"] = base_date + pd.to_timedelta(df_gantt_3["arr_time_min"], unit="m")
    df_gantt_3['energy_required'] = df_gantt_3['arr_time_min'] - df_gantt_3['dep_time_min']
    df_gantt_3['id'] = df_gantt_3['bus_id'].apply(lambda x: f"Bus{x}")

    ########################################## 1 CS ##############################################
    # Create base color map dynamically
    trip_ids = df_gantt["trip_id"].unique()
    color_map = {t: "#1f77b4" for t in trip_ids}  # default all blue

    # Override "CS*" trip colors
    for t in trip_ids:
        if str(t).startswith("CS"):
            color_map[t] = "#ff7f0e"

    fig_gantt = px.timeline(
        df_gantt, 
        x_start="dep_time", x_end="arr_time", 
        y="id", color="trip_id",
        title=f"Electric Bus Timetable {case}1CS",
        color_discrete_map=color_map
    )

    fig_gantt.update_layout(
        yaxis=dict(
            tickmode="array",
            tickvals=df_gantt.id.unique(),
            ticktext=[f"{i}" for i in df_gantt.id.unique()]
        )
    )

    fig_gantt.update_yaxes(autorange="reversed")

    ########################################## 3 CS ##############################################
    trip_ids = df_gantt_3["trip_id"].unique()
    color_map = {t: "#1f77b4" for t in trip_ids}  # default all blue

    # Override "CS*" trip colors
    for t in trip_ids:
        if str(t).startswith("CS"):
            color_map[t] = "#ff7f0e"

    fig_gantt_3 = px.timeline(
        df_gantt_3, 
        x_start="dep_time", x_end="arr_time", 
        y="id", color="trip_id",
        title=f"Electric Bus Timetable {case}3CS",
        color_discrete_map=color_map
    )

    fig_gantt_3.update_layout(
        yaxis=dict(
            tickmode="array",
            tickvals=df_gantt_3.id.unique(),
            ticktext=[f"{i}" for i in df_gantt_3.id.unique()]
        )
    )

    fig_gantt_3.update_yaxes(autorange="reversed")
    return fig_gantt, fig_gantt_3,

# 2️⃣ Update SoC Line Chart based on clicks from either Gantt
@app.callback(
    Output("soc_line", "figure"),
    Output("soc_line_3", "figure"),
    [Input("gantt", "clickData"),
    Input("gantt_3cs", "clickData"),
    Input("instance_selector", "value")]
)
def show_soc_line(clickData, clickData3cs, case):
    triggered = ctx.triggered_id
    work_dir = f"./FINAL_TimeLimit/{case}rips/"
    single = pd.read_csv(f"{work_dir}CHSSA_{case}1CS_df.csv")
    df_gantt = single.drop(columns="Unnamed: 0")
    df_gantt['dep_time_min'] = df_gantt['dep_time']
    df_gantt['arr_time_min'] = df_gantt['arr_time']
    df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
    df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
    df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
    df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

    many = pd.read_csv(f"{work_dir}CHSSA_{case}3CS_df.csv")
    df_gantt_3 = many.drop(columns="Unnamed: 0")
    df_gantt_3.head()
    df_gantt_3['dep_time_min'] = df_gantt_3['dep_time']
    df_gantt_3['arr_time_min'] = df_gantt_3['arr_time']
    df_gantt_3["dep_time"] = base_date + pd.to_timedelta(df_gantt_3["dep_time_min"], unit="m")
    df_gantt_3["arr_time"] = base_date + pd.to_timedelta(df_gantt_3["arr_time_min"], unit="m")
    df_gantt_3['energy_required'] = df_gantt_3['arr_time_min'] - df_gantt_3['dep_time_min']
    df_gantt_3['id'] = df_gantt_3['bus_id'].apply(lambda x: f"Bus{x}")
    
    soc_data = []
    for bus_id, group in df_gantt.groupby("bus_id"):
        soc = 100
        for id, trip in group.iterrows():
            trip_id = trip['trip_id']
            if trip_id.startswith("CS"):
                soc = 0
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
                soc_data.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, 100, len(time_range))
                }))
                soc = 100
            else:
                prev_soc = soc
                soc = soc - (trip['energy_required']/D_MAX * 100)
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
                soc_data.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
                }))
    df_soc = pd.concat(soc_data)


    soc_data_3 = []
    for bus_id, group in df_gantt_3.groupby("bus_id"):
        soc = 100
        for id, trip in group.iterrows():
            trip_id = trip['trip_id']
            if trip_id.startswith("CS"):
                soc = 0
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
                soc_data_3.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, 100, len(time_range))
                }))
                soc = 100
            else:
                prev_soc = soc
                soc = soc - (trip['energy_required']/D_MAX * 100)
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
                soc_data_3.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
                }))

    df_soc_3 = pd.concat(soc_data_3)  
    
    bus_clicked = bus3_clicked = None
    if triggered == "gantt" and clickData:
        bus_clicked = clickData["points"][0]["y"]
    elif triggered == "gantt_3cs" and clickData3cs:
        bus3_clicked = clickData3cs["points"][0]["y"]
    else:
        print(f"CLIKED DATA = {clickData}")
        fig = px.line(df_soc, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
        fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
        fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
        
        fig_3 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
        fig_3.update_yaxes(title="State of Charge (%)", range=[0, 100])
        fig_3.update_xaxes(title="Time of Day", tickformat="%H:%M") 
        return fig, fig_3
    
    if bus_clicked:
        df_selected = df_soc[df_soc["bus_id"] == bus_clicked]
        df_selected_3 = df_soc_3[df_soc_3["bus_id"] == bus_clicked]
    elif bus3_clicked:
        df_selected = df_soc[df_soc["bus_id"] == bus3_clicked]
        df_selected_3 = df_soc_3[df_soc_3["bus_id"] == bus3_clicked]
        bus_clicked = bus3_clicked
     
    
    fig = px.line(df_soc, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
    fig2 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
    fig2.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig2.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
    clickData = clickData3cs = None
    return fig, fig2

app.run(debug=True)

CASE!!! = 10T
CLIKED DATA = None


CASE!!! = 10T
CLIKED DATA = None
CASE!!! = 10T
CASE!!! = 10T
CASE!!! = 10T
CLIKED DATA = None
CLIKED DATA = None
CLIKED DATA = None
CASE!!! = 40T
---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
~\AppData\Local\Temp\ipykernel_1360\3320437646.py in test_instances(case='40T')
     65     print(f"CASE!!! = {case}")
     66     work_dir = f"./FINAL_TimeLimit/{case}rips/"
---> 67     single = pd.read_csv(f"{work_dir}CHSSA_{case}1CS_df.csv")
        single = undefined
        global pd.read_csv = <function read_csv at 0x00000221D8E44790>
     68     df_gantt = single.drop(columns="Unnamed: 0")
     69     df_gantt['dep_time_min'] = df_gantt['dep_time']

~\.conda\envs\backup_envo\lib\site-packages\pandas\util\_decorators.py in wrapper(
    *args=('./FINAL_TimeLimit/40Trips/CHSSA_40T1CS_df.csv',),
    **kwargs={}
)
    209                 else:
    210                     kwargs[new_arg_name]

CASE!!! = 54T
CLIKED DATA = None
CASE!!! = 10T
CLIKED DATA = None
CASE!!! = 30T
CLIKED DATA = {'points': [{'curveNumber': 5, 'pointNumber': 0, 'pointIndex': 0, 'x': '2025-01-01 13:35', 'y': 'Bus2', 'label': 'Bus2', 'value': '2025-01-01 13:35', 'base': '2025-01-01T11:15:00', 'bbox': {'x0': 702.96, 'x1': 702.96, 'y0': 282, 'y1': 354}}]}


In [ ]:
df_gantt

In [ ]:
df_gantt_3

In [55]:
app = Dash(__name__)

case = "54T"
work_dir = f"./FINAL_TimeLimit/{case}rips/"
single = pd.read_csv(f"{work_dir}CHSSA_{case}1CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

many = pd.read_csv(f"{work_dir}CHSSA_{case}3CS_df.csv")
df_gantt_3 = many.drop(columns="Unnamed: 0")
df_gantt_3.head()
df_gantt_3['dep_time_min'] = df_gantt_3['dep_time']
df_gantt_3['arr_time_min'] = df_gantt_3['arr_time']
df_gantt_3["dep_time"] = base_date + pd.to_timedelta(df_gantt_3["dep_time_min"], unit="m")
df_gantt_3["arr_time"] = base_date + pd.to_timedelta(df_gantt_3["arr_time_min"], unit="m")
df_gantt_3['energy_required'] = df_gantt_3['arr_time_min'] - df_gantt_3['dep_time_min']
df_gantt_3['id'] = df_gantt_3['bus_id'].apply(lambda x: f"Bus{x}")
soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)


soc_data_3 = []
for bus_id, group in df_gantt_3.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        print(f"busid = {bus_id}")
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data_3.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data_3.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))

df_soc_3 = pd.concat(soc_data_3)

########################################## 1 CS ##############################################
# Create base color map dynamically
trip_ids = df_gantt["trip_id"].unique()
color_map = {t: "#1f77b4" for t in trip_ids}  # default all blue

# Override "CS*" trip colors
for t in trip_ids:
    if str(t).startswith("CS"):
        color_map[t] = "#ff7f0e"

fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="id", color="trip_id",
    title=f"Electric Bus Timetable {case}1CS",
    color_discrete_map=color_map
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

fig_soc = px.line(df_soc, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
fig_soc.update_yaxes(title="State of Charge (%)", range=[0, 100])
fig_soc.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
########################################## 3 CS ##############################################
trip_ids = df_gantt_3["trip_id"].unique()
color_map = {t: "#1f77b4" for t in trip_ids}  # default all blue

# Override "CS*" trip colors
for t in trip_ids:
    if str(t).startswith("CS"):
        color_map[t] = "#ff7f0e"
        
fig_gantt_3 = px.timeline(
    df_gantt_3, 
    x_start="dep_time", x_end="arr_time", 
    y="id", color="trip_id",
    title=f"Electric Bus Timetable {case}3CS",
    color_discrete_map=color_map
)

fig_gantt_3.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc_3.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc_3.bus_id.unique()]
    )
)

fig_gantt_3.update_yaxes(autorange="reversed")

fig_soc_3 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
fig_soc_3.update_yaxes(title="State of Charge (%)", range=[0, 100])
fig_soc_3.update_xaxes(title="Time of Day", tickformat="%H:%M")

title_components = [
    html.H3(f"Electric Bus Rostering Dashboard {case}Trips1CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.H3(f"Electric Bus Rostering Dashboard {case}Trips3CS"),
    dcc.Graph(id="gantt_3cs", figure=fig_gantt_3),
    html.Hr(),
    html.H4("1CS Battery State of Charge (SoC)"),
    dcc.Graph(id="soc_line", figure=fig_soc),
    html.H4("3CS Battery State of Charge (SoC)"),
    dcc.Graph(id="soc_line_3", figure=fig_soc_3)
]

app.layout = html.Div(title_components)

@app.callback(
    Output("soc_line", "figure"),
    Output("soc_line_3", "figure"),
    [Input("gantt", "clickData"),
    Input("gantt_3cs", "clickData")]
)
def show_soc_line(clickData, clickData3cs):
    triggered = ctx.triggered_id
    bus_clicked = bus3_clicked = None
    if triggered == "gantt" and clickData:
        bus_clicked = clickData["points"][0]["y"]
    elif triggered == "gantt_3cs" and clickData3cs:
        bus3_clicked = clickData3cs["points"][0]["y"]
    else:
        fig = px.line(df_soc, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
        fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
        fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
        
        fig_3 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
        fig_3.update_yaxes(title="State of Charge (%)", range=[0, 100])
        fig_3.update_xaxes(title="Time of Day", tickformat="%H:%M") 
        return fig, fig_3
    
    if bus_clicked:
        df_selected = df_soc[df_soc["bus_id"] == bus_clicked]
        df_selected_3 = df_soc_3[df_soc_3["bus_id"] == bus_clicked]
    elif bus3_clicked:
        df_selected = df_soc[df_soc["bus_id"] == bus3_clicked]
        df_selected_3 = df_soc_3[df_soc_3["bus_id"] == bus3_clicked]
        bus_clicked = bus3_clicked

    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
    fig2 = px.line(df_selected_3, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title=f"Bus{bus_clicked}: State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    clickData = clickData3cs = None
    return fig, fig2


app.run(debug=True)

busid = 1
busid = 1
busid = 2
busid = 2
busid = 3
busid = 3
busid = 4
busid = 5
busid = 5
busid = 6
busid = 6
busid = 6
busid = 6
busid = 7
busid = 7
busid = 7
busid = 7
busid = 7
busid = 7
busid = 7
busid = 8
busid = 8
busid = 8
busid = 8
busid = 8
busid = 8
busid = 9
busid = 9
busid = 9
busid = 9
busid = 9
busid = 9
busid = 10
busid = 10
busid = 10
busid = 10
busid = 10
busid = 11
busid = 11
busid = 11
busid = 11
busid = 11
busid = 12
busid = 12
busid = 12
busid = 12
busid = 12
busid = 13
busid = 13
busid = 13
busid = 13
busid = 13
busid = 14
busid = 14
busid = 14
busid = 14
busid = 15
busid = 15
busid = 15
busid = 15
busid = 16
busid = 16
busid = 16
busid = 16
busid = 16


In [54]:
yaxis=dict(
    tickmode="array",
    tickvals=df_soc.bus_id.unique(),
    ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
)
yaxis

{'tickmode': 'array',
 'tickvals': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
       dtype=int64),
 'ticktext': ['Bus1',
  'Bus2',
  'Bus3',
  'Bus4',
  'Bus5',
  'Bus6',
  'Bus7',
  'Bus8',
  'Bus9',
  'Bus10',
  'Bus11',
  'Bus12',
  'Bus13',
  'Bus14',
  'Bus15',
  'Bus16']}

In [35]:
df_soc_3

,bus_id,time,soc
0,11,2025-01-01 09:00:00,100.000000
1,11,2025-01-01 09:10:00,97.032967
2,11,2025-01-01 09:20:00,94.065934
3,11,2025-01-01 09:30:00,91.098901
4,11,2025-01-01 09:40:00,88.131868
...,...,...,...
9,12,2025-01-01 19:50:00,34.725275
10,12,2025-01-01 20:00:00,31.758242
11,12,2025-01-01 20:10:00,28.791209
12,12,2025-01-01 20:20:00,25.824176


In [22]:
df_gantt_3.loc[df_gantt_3.bus_id.isin([11,13])]

,bus_id,trip_id,dep_time,arr_time,duration,dep_terminal,arr_terminal,next_dep,difference,dep_time_min,arr_time_min,energy_required,id
37,11,BA1,2025-01-01 09:00:00,2025-01-01 11:15:00,135,Terminal B,Terminal A,790.0,115.0,540,675,135,Bus11
38,11,AD3,2025-01-01 13:10:00,2025-01-01 15:25:00,135,Terminal A,Terminal D,975.0,50.0,790,925,135,Bus11
39,11,CS1,2025-01-01 15:25:00,2025-01-01 16:15:00,75,Terminal D,Terminal D,965.0,0.0,925,975,50,Bus11
40,11,DC4,2025-01-01 16:05:00,2025-01-01 18:20:00,135,Terminal D,Terminal C,1120.0,20.0,965,1100,135,Bus11
41,11,AD5,2025-01-01 18:40:00,2025-01-01 20:40:00,140,Terminal A,Terminal D,0.0,0.0,1120,1240,120,Bus11
47,13,AD1,2025-01-01 09:20:00,2025-01-01 11:40:00,140,Terminal A,Terminal D,740.0,40.0,560,700,140,Bus13
48,13,CA2,2025-01-01 12:20:00,2025-01-01 14:40:00,140,Terminal C,Terminal A,930.0,50.0,740,880,140,Bus13
49,13,CS1,2025-01-01 14:40:00,2025-01-01 15:30:00,75,Terminal A,Terminal A,920.0,0.0,880,930,50,Bus13
50,13,CA4,2025-01-01 15:20:00,2025-01-01 17:40:00,140,Terminal C,Terminal A,1080.0,20.0,920,1060,140,Bus13
51,13,BA7,2025-01-01 18:00:00,2025-01-01 20:15:00,135,Terminal B,Terminal A,0.0,0.0,1080,1215,135,Bus13


In [15]:
soc_data_3 = []
for bus_id, group in df_gantt_3.groupby("bus_id"):
    soc = 100
    if bus_id == 11 or bus_id == 12:
        for idx, trip in group.iterrows():
            trip_id = trip['trip_id']
            if trip_id.startswith("CS"):
                print(trip)
                soc = 0
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
                soc_data_3.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, 100, len(time_range))
                }))
                soc = 100
                print(f"CS also = {bus_id}\n{time_range}")
                print(f"CS!!! = {soc_data_3}")
            else:
                prev_soc = soc
                soc = soc - (trip['energy_required']/D_MAX * 100)
                time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
                soc_data_3.append(pd.DataFrame({
                    "bus_id": bus_id,
                    "time": time_range,
                    "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
                }))

df_soc_3 = pd.concat(soc_data_3)
df_soc_3

bus_id                              11
trip_id                            CS1
dep_time           2025-01-01 15:25:00
arr_time           2025-01-01 16:15:00
duration                            75
dep_terminal                Terminal D
arr_terminal                Terminal D
next_dep                         965.0
difference                         0.0
dep_time_min                       925
arr_time_min                       975
energy_required                     50
id                               Bus11
Name: 39, dtype: object
CS also = 11
DatetimeIndex(['2025-01-01 15:25:00', '2025-01-01 15:35:00',
               '2025-01-01 15:45:00', '2025-01-01 15:55:00',
               '2025-01-01 16:05:00'],
              dtype='datetime64[ns]', freq='10T')
CS!!! = [    bus_id                time         soc
0       11 2025-01-01 09:00:00  100.000000
1       11 2025-01-01 09:10:00   97.032967
2       11 2025-01-01 09:20:00   94.065934
3       11 2025-01-01 09:30:00   91.098901
4       11 2025-01-01

,bus_id,time,soc
0,11,2025-01-01 09:00:00,100.000000
1,11,2025-01-01 09:10:00,97.032967
2,11,2025-01-01 09:20:00,94.065934
3,11,2025-01-01 09:30:00,91.098901
4,11,2025-01-01 09:40:00,88.131868
...,...,...,...
9,12,2025-01-01 19:50:00,34.725275
10,12,2025-01-01 20:00:00,31.758242
11,12,2025-01-01 20:10:00,28.791209
12,12,2025-01-01 20:20:00,25.824176


In [16]:
df_gantt_3.loc[df_gantt_3.trip_id == "CS1a"]

,bus_id,trip_id,dep_time,arr_time,duration,dep_terminal,arr_terminal,next_dep,difference,dep_time_min,arr_time_min,energy_required,id
44,12,CS1a,2025-01-01 14:15:00,2025-01-01 15:05:00,75,Terminal A,Terminal A,920.0,15.0,855,905,50,Bus12
62,16,CS1a,2025-01-01 15:40:00,2025-01-01 16:30:00,75,Terminal A,Terminal A,975.0,0.0,940,990,50,Bus16


work_dir = r"./FINAL_TimeLimit/10Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_10T3CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()

In [ ]:

app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/10Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_10T3CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 10Trips3CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

## 20Trips

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/20Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_20T1CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

many = pd.read_csv(f"{work_dir}CHSSA_{case}3CS_df.csv")
df_gantt_3 = many.drop(columns="Unnamed: 0")
df_gantt_3.head()
df_gantt_3['dep_time_min'] = df_gantt_3['dep_time']
df_gantt_3['arr_time_min'] = df_gantt_3['arr_time']
df_gantt_3["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt_3["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt_3['energy_required'] = df_gantt_3['arr_time_min'] - df_gantt_3['dep_time_min']
df_gantt_3['id'] = df_gantt_3['bus_id'].apply(lambda x: f"Bus{x}")
soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)


soc_data_3 = []
for bus_id, group in df_gantt_3.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data_3.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data_3.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc_3 = pd.concat(soc_data_3)

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title=f"Electric Bus Timetable {case}1CS"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

fig_soc = px.line(df_soc, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
fig_soc.update_yaxes(title="State of Charge (%)", range=[0, 100])
fig_soc.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
########################################## 3 CS ##############################################
fig_gantt_3 = px.timeline(
    df_gantt_3, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title=f"Electric Bus Timetable {case}3CS"
)

fig_gantt_3.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc_3.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc_3.bus_id.unique()]
    )
)

fig_gantt_3.update_yaxes(autorange="reversed")

fig_soc_3 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
fig_soc_3.update_yaxes(title="State of Charge (%)", range=[0, 100])
fig_soc_3.update_xaxes(title="Time of Day", tickformat="%H:%M")

title_components = [
    html.H3(f"Electric Bus Rostering Dashboard {case}Trips1CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.H3(f"Electric Bus Rostering Dashboard {case}Trips3CS"),
    dcc.Graph(id="gantt_3cs", figure=fig_gantt_3),
    html.Hr(),
    html.H4("1CS Battery State of Charge (SoC)"),
    dcc.Graph(id="soc_line", figure=fig_soc),
    html.H4("3CS Battery State of Charge (SoC)"),
    dcc.Graph(id="soc_line_3", figure=fig_soc_3)
]

app.layout = html.Div(title_components)

@app.callback(
    Output("soc_line", "figure"),
    Output("soc_line_3", "figure"),
    [Input("gantt", "clickData"),
    Input("gantt_3cs", "clickData")]
)
def show_soc_line(clickData, clickData3cs):
    triggered = ctx.triggered_id
    bus_clicked = bus3_clicked = None
    if triggered == "gantt" and clickData:
        bus_clicked = clickData["points"][0]["y"]
    elif triggered == "gantt_3cs" and clickData3cs:
        bus3_clicked = clickData3cs["points"][0]["y"]
    else:
        fig = px.line(df_soc, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
        fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
        fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
        
        fig_3 = px.line(df_soc_3, x="time", y="soc", color="bus_id", title="State of Charge (All Buses)")
        fig_3.update_yaxes(title="State of Charge (%)", range=[0, 100])
        fig_3.update_xaxes(title="Time of Day", tickformat="%H:%M") 
        return fig, fig_3
    
    if bus_clicked:
        df_selected = df_soc[df_soc["bus_id"] == bus_clicked]
        df_selected_3 = df_soc_3[df_soc_3["bus_id"] == bus_clicked]
    elif bus3_clicked:
        df_selected = df_soc[df_soc["bus_id"] == bus3_clicked]
        df_selected_3 = df_soc_3[df_soc_3["bus_id"] == bus3_clicked]
        bus_clicked = bus3_clicked

    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    
    fig2 = px.line(df_selected_3, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title=f"Bus{bus_clicked}: State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    clickData = clickData3cs = None
    return fig, fig2


app.run(debug=True)

In [5]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/20Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_20T1CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 20Trips1CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

components.append(components.append(dcc.Graph(id=f"soc_line")))
# for bid, fig in enumerate(figs, start=1):
#     components.append(html.H4("Battery State of Charge (SoC)"))
#     components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# Callback: when user clicks a trip block on the Gantt
@app.callback(
    Output("soc_line", "figure"),
    Input("gantt", "clickData")
)
def show_soc_line(clickData):
    if not clickData:
        return go.Figure()

    # Get clicked bus_id
    bus_clicked = clickData["points"][0]["y"]

    # Filter the SoC data
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

    return fig


app.run(debug=True)

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/20Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_20T3CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 20Trips3CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

## 30Trips

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/30Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_30T1CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 30Trips1CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/30Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_30T3CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 30Trips3CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

## 40Trips

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/40Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_40T1CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 40Trips1CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/40Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_40T3CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 40Trips3CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

## 54Trips

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/54Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_54T1CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 54Trips1CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/54Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_54T3CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 54Trips3CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

## 100Trips

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/100Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_100T1CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 100Trips1CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)

In [ ]:
app = Dash(__name__)


work_dir = r"./FINAL_TimeLimit/100Trips/"
single = pd.read_csv(f"{work_dir}CHSSA_100T3CS_df.csv")
df_gantt = single.drop(columns="Unnamed: 0")
df_gantt.head()
df_gantt['dep_time_min'] = df_gantt['dep_time']
df_gantt['arr_time_min'] = df_gantt['arr_time']
df_gantt["dep_time"] = base_date + pd.to_timedelta(df_gantt["dep_time_min"], unit="m")
df_gantt["arr_time"] = base_date + pd.to_timedelta(df_gantt["arr_time_min"], unit="m")
df_gantt['energy_required'] = df_gantt['arr_time_min'] - df_gantt['dep_time_min']
df_gantt['id'] = df_gantt['bus_id'].apply(lambda x: f"Bus{x}")

soc_data = []
for bus_id, group in df_gantt.groupby("bus_id"):
    soc = 100
    for id, trip in group.iterrows():
        trip_id = trip['trip_id']
        if trip_id.startswith("CS"):
            soc = 0
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'] - pd.Timedelta("10m"), freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, 100, len(time_range))
            }))
            soc = 100
        else:
            prev_soc = soc
            soc = soc - (trip['energy_required']/D_MAX * 100)
            time_range = pd.date_range(trip['dep_time'], trip['arr_time'], freq="10min")
            soc_data.append(pd.DataFrame({
                "bus_id": bus_id,
                "time": time_range,
                "soc": np.linspace(soc, prev_soc, len(time_range))[::-1]
            }))
df_soc = pd.concat(soc_data)

print(f"SOC={df_soc.head(20)}")

# Create the Gantt chart
fig_gantt = px.timeline(
    df_gantt, 
    x_start="dep_time", x_end="arr_time", 
    y="bus_id", color="trip_id",
    title="Electric Bus Timetable"
)

fig_gantt.update_layout(
    yaxis=dict(
        tickmode="array",
        tickvals=df_soc.bus_id.unique(),
        ticktext=[f"Bus{i}" for i in df_soc.bus_id.unique()]
    )
)

fig_gantt.update_yaxes(autorange="reversed")

figs = []
for bus_clicked in df_soc.bus_id.unique():
    df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

    # Create line chart
    fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
    fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
    fig.update_xaxes(title="Time of Day", tickformat="%H:%M")
    figs.append(fig)

components = [html.H3("Electric Bus Rostering Dashboard 100Trips3CS"),
    dcc.Graph(id="gantt", figure=fig_gantt),
    html.Hr()
]

for bid, fig in enumerate(figs, start=1):
    components.append(html.H4("Battery State of Charge (SoC)"))
    components.append(dcc.Graph(id=f"soc_line{bid}", figure=fig))
    


# Layout
app.layout = html.Div(components)

# # Callback: when user clicks a trip block on the Gantt
# @app.callback(
#     Output("soc_line", "figure"),
#     Input("gantt", "clickData")
# )
# def show_soc_line(clickData):
#     if not clickData:
#         return go.Figure()

#     # Get clicked bus_id
#     bus_clicked = clickData["points"][0]["y"]

#     # Filter the SoC data
#     df_selected = df_soc[df_soc["bus_id"] == bus_clicked]

#     # Create line chart
#     fig = px.line(df_selected, x="time", y="soc", title=f"SoC over Time - {bus_clicked}")
#     fig.update_yaxes(title="State of Charge (%)", range=[0, 100])
#     fig.update_xaxes(title="Time of Day", tickformat="%H:%M")

#     return fig


app.run(debug=True)